# Getting the final GTF

After expanding the GTF with StringTie and MicroExonator, we further process the obtained GTF to:

1.- Make sure that all new transcripts are correctly assinged to genes
2.- All genes has at least 3 exons 
3.- Collapse begining and ends of genes

In [ ]:
import csv
from collections import defaultdict

In [ ]:
exon_gene = dict()
gene_exons = defaultdict(set)
transcript_exons = defaultdict(list)


with open("/lustre/scratch117/cellgen/team218/gp7/Joe/MicroExonator/Report/out.high_quality.gtf") as file:
    
    for row in csv.reader(file, delimiter = '\t'):

        if row[0][0]!="#":

            chrom = row[0]
            source = row[1]
            strand =  row[6]
            feature = row[2]
            start = row[3]
            end = row[4]

            tags = row[8].strip(";").split("; ")

            tag_dict = dict()

            for t in tags:
                tag = t.strip(" ")
                if len(tag.split(' '))==2:
                    field, value = tag.split(' ')
                    value = value.strip('"')
                    tag_dict[field] = value

            gene_id = tag_dict["gene_id"]
 
            if feature=="exon":
                gene_exons[gene_id].add((chrom, strand, start, end))
            
                transcript_exons[tag_dict["transcript_id"]].append((chrom, strand, start, end))
                
                if "ENSMUS" in gene_id:
                        exon_gene[(chrom, strand, start, end)] = gene_id
                        
                        
                        



In [ ]:
gene_re_annotation = dict()

with open("/lustre/scratch117/cellgen/team218/gp7/Joe/MicroExonator/Report/out.high_quality.gtf") as file:
    
    for row in csv.reader(file, delimiter = '\t'):

        if row[0][0]!="#":

            chrom = row[0]
            source = row[1]
            strand =  row[6]
            feature = row[2]
            start = row[3]
            end = row[4]

            tags = row[8].strip(";").split("; ")

            tag_dict = dict()

            for t in tags:
                tag = t.strip(" ")
                if len(tag.split(' '))==2:
                    field, value = tag.split(' ')
                    value = value.strip('"')
                    tag_dict[field] = value

            gene_id = tag_dict["gene_id"]
 
            if "ENSMUS" not in gene_id:
        
                if feature=="exon":
                
                    exon = (chrom, strand, start, end)
                    
                    if exon in exon_gene:
                
                        new_gene_id = exon_gene[exon]
                    
                        gene_re_annotation[gene_id] = new_gene_id
                



In [ ]:
gene_t_starts = defaultdict(list)
gene_t_ends = defaultdict(list)

with open("/lustre/scratch117/cellgen/team218/gp7/Joe/MicroExonator/Report/out.high_quality.gtf") as file:
    
    for row in csv.reader(file, delimiter = '\t'):

        if row[0][0]!="#":

            chrom = row[0]
            source = row[1]
            strand =  row[6]
            feature = row[2]
            start = row[3]
            end = row[4]

            tags = row[8].strip(";").split("; ")

            tag_dict = dict()

            for t in tags:
                tag = t.strip(" ")
                if len(tag.split(' '))==2:
                    field, value = tag.split(' ')
                    value = value.strip('"')
                    tag_dict[field] = value

            gene_id = tag_dict["gene_id"]

            
            
            if gene_id in gene_re_annotation:
                gene_id = gene_re_annotation[gene_id]
            
                        
            if feature=="transcript":
                
                gene_t_starts[gene_id].append(int(start))
                gene_t_ends[gene_id].append(int(end))

In [ ]:
window_t_starts = []
window_t_ends = []

gene_round_starts = dict()
gene_round_ends = dict()

for gene, starts in gene_t_starts.items():
    
    start_bins = defaultdict(list)
    round_starts = dict()
    
    for s in starts:
        start_round = str(s)[:-2]
        start_bins[start_round].append(s)
        
    for start_bin, start_list in start_bins.items():
        round_starts[start_bin] =  max(set(start_list), key = start_list.count)
        
    gene_round_starts[gene]  = round_starts
    
        
for gene, ends in gene_t_ends.items():
    
    end_bins = defaultdict(list)
    round_ends = dict()
    
    for e in ends:
        end_round = str(e)[:-2]
        end_bins[end_round].append(e)
        
    for end_bin, end_list in end_bins.items():
        round_ends[end_bin] =  max(set(end_list), key = end_list.count)
        
    gene_round_ends[gene]  = round_ends
    
    

In [ ]:
for gene, starts in gene_t_starts.items():
    print(gene, sorted(starts))

In [ ]:

transcript_first_exon = dict()
transcript_last_exon = dict()

for transcript, exons in transcript_exons.items():
    transcript_first_exon[transcript] = exons[0]
    transcript_last_exon[transcript] = exons[-1]
    


In [82]:
out = open("/lustre/scratch117/cellgen/team218/gp7/Joe/MicroExonator/Report/out.high_quality.calibrated.gtf", "w") 

writer = csv.writer(out, delimiter="\t", quotechar='', quoting=csv.QUOTE_NONE, escapechar='\\')


with open("/lustre/scratch117/cellgen/team218/gp7/Joe/MicroExonator/Report/out.high_quality.gtf") as file:
    
    for row in csv.reader(file, delimiter = '\t'):

        if row[0][0]!="#":

            chrom = row[0]
            source = row[1]
            strand =  row[6]
            feature = row[2]
            start = row[3]
            end = row[4]

            tags = row[8].strip(";").split("; ")

            tag_dict = dict()

            for t in tags:
                tag = t.strip(" ")
                if len(tag.split(' '))==2:
                    field, value = tag.split(' ')
                    value = value.strip('"')
                    tag_dict[field] = value

            gene_id = tag_dict["gene_id"]
            
            if gene_id in gene_re_annotation:
                gene_id = gene_re_annotation[gene_id]
                
            tag_dict["gene_id"] = gene_id

            nexons = len(gene_exons[tag_dict["gene_id"]])
   
            if nexons>=3:
        
        
                if feature=="exon":
                
                    transcript_id = tag_dict["transcript_id"]
                    exon = (chrom, strand, start, end)
                    
                    
                
                    if gene_id in gene_round_starts and start[:-2] in gene_round_starts[gene_id] and exon==transcript_first_exon[transcript_id]:

                        start = gene_round_starts[gene_id][start[:-2]]

                    if gene_id in gene_round_ends and end[:-2] in gene_round_ends[gene_id] and exon==transcript_last_exon[transcript_id]:

                        end = gene_round_ends[gene_id][end[:-2]]

                    
                    if int(start)>int(end):
                        
                        start = exon[-2]
                        end = exon[-1]
                        
                    row[3] = start
                    row[4] = end
                        
                new_tags_list = []

                for field, value in tag_dict.items():

                    pair = field + ' "' + value + '";'
                    new_tags_list.append(pair) 

                new_tags = " ".join(new_tags_list)
                new_line = row[:-1] + [new_tags]
                
                #print(new_line)

                #rint(new_line)

                writer.writerow(new_line)
                    #print(transcript_id)
            

In [ ]:
[1,2,3,1,3,2,1].count(10)